In [1]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist

import re
import pandas as pd
import time

In [4]:
!which chromedriver

/usr/local/bin/chromedriver


In [2]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
#Getting news article from nasa.gov
nasa_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(nasa_url)
time.sleep(2)



In [17]:
html = browser.html

soup = bs(html, 'html.parser')

news = soup.find('ul',class_='item_list').find('li', class_='slide')

title = news.find('div', class_='content_title').text
#title
news_content = news.find('div', class_='article_teaser_body').text
#news_content

'The first helicopter attempting to fly on another planet is a marvel of engineering. Get up to speed with these key facts about its plans.'

In [22]:
#Getting featured photo from JPL

url_jpl = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url_jpl)
time.sleep(2)

In [23]:
html = browser.html

soup= bs(html, 'html.parser')

browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(2)

In [30]:
html = browser.html
soup = bs(html, 'html.parser')

image_path = soup.find('img', class_='fancybox-image')['src']

image_url = 'https://www.jpl.nasa.gov' + image_path

#image_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19346_ip.jpg'

In [42]:
#Twitter Scrape


# URL of page to be scraped
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)
time.sleep(2)

In [43]:

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

weather = soup.find('span', text = re.compile('InSight sol')).text


weather = weather.replace('InSight s', 'S') 
weather

'Sol 582 (2020-07-16) low -89.1ºC (-128.4ºF) high -4.6ºC (23.8ºF)\npressure at 7.80 hPa'

In [44]:
#Mars Facts

facts_url = 'https://space-facts.com/mars/'
browser.visit(facts_url)
time.sleep(2)

In [55]:
html = browser.html

table = pd.read_html(html)
#table

mars_facts_df = pd.DataFrame(table[0])
mars_facts_df = mars_facts_df.rename(columns = {0: 'Description', 1: 'Value'})
mars_facts_df = mars_facts_df.set_index('Description')

mars_facts_df.index.name=None

mars_facts_htmltable = mars_facts_df.to_html(header =False)
mars_facts_htmltable = mars_facts_htmltable.replace('\n','')
#mars_facts_htmltable

'<table border="1" class="dataframe">  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [96]:
#Hemisphere Images Scrape
hemi_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemi_url)
time.sleep(2)

In [97]:
html = browser.html

soup = bs(html,'html.parser')
#soup

hemi_names = []
hemi_urls = []
count = 0


result = soup.find_all('a', class_='itemLink product-item')
result


for a in result:
    try:
        hemi_name=a.find('h3').text
        hemi_names.append(hemi_name)
    except:
        count +=1
        
#hemi_names

for name in hemi_names:
    
    time.sleep(2)
    
    hemi_dict = {}
    
    browser.click_link_by_partial_text(name)
    time.sleep(2)
    
    html = browser.html
    soup = bs(html, 'html.parser')
    
    image_path = soup.find('img', class_='wide-image')['src']
    
    image_url = 'https://astrogeology.usgs.gov' + image_path
    
    hemi_dict['Title'] = name
    hemi_dict['url']=image_url
    
    hemi_urls.append(hemi_dict)
    
    browser.back()
    
print(hemi_names, hemi_urls)

['Cerberus Hemisphere Enhanced', 'Schiaparelli Hemisphere Enhanced', 'Syrtis Major Hemisphere Enhanced', 'Valles Marineris Hemisphere Enhanced'] [{'Title': 'Cerberus Hemisphere Enhanced', 'url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'Title': 'Schiaparelli Hemisphere Enhanced', 'url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'Title': 'Syrtis Major Hemisphere Enhanced', 'url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'Title': 'Valles Marineris Hemisphere Enhanced', 'url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]


In [98]:
meta_data = {}

meta_data['title'] = title
meta_data['content']= news_content
meta_data['featured_image']= image_url
meta_data['weather'] = weather
meta_data['facts']=mars_facts_htmltable
meta_data['hemisphere_images']=hemi_urls

In [99]:
meta_data

{'title': "6 Things to Know About NASA's Ingenuity Mars Helicopter",
 'content': 'The first helicopter attempting to fly on another planet is a marvel of engineering. Get up to speed with these key facts about its plans.',
 'featured_image': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg',
 'weather': 'Sol 582 (2020-07-16) low -89.1ºC (-128.4ºF) high -4.6ºC (23.8ºF)\npressure at 7.80 hPa',
 'facts': '<table border="1" class="dataframe">  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      